In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import torchvision
import torchvision.transforms as transforms

IMAGE_WIDTH = 244
IMAGE_HEIGHT = 244
EPOCHS = 3


def main():

    # Prepare data
    train_path = os.path.join("../input/skin-cancer-malignant-vs-benign", "train")
    test_path = os.path.join("../input/skin-cancer-malignant-vs-benign", "test")

    transform = transforms.Compose([
        # More transformations can be added
        transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH), ),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # print("Data Loading...")
    train_data = datasets.ImageFolder(root=train_path, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
    test_data = datasets.ImageFolder(root=test_path, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=True)
    # print("Data loaded")

    # Train Neural Net
    net = Net()
    print(net)
    # print(len(train_loader), "Train loader length")
    # print(len(test_loader), "test Loader Length")
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.0015, momentum=0.9)
    
    
    for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            # print("helllooo")
            evidence, labels = data
            # print(evidence.shape, " Evidence Shape")
            # print(labels.shape, " Label Shape")
            break
            
    count = 0

    
    for epoch in range(EPOCHS):  # loop over the dataset multiple times
        print("Beginning training ", epoch+1)
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # break
            # print statistics
            running_loss += loss.item()
            if i % 100 == 0 and i != 0:  # print every 100 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 200))
                running_loss = 0.0
        # break
    print('Finished Training')

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            evidence, labels = data
            outputs = net(evidence)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(correct, " correct")
    print(total, " total")
    print('Accuracy of the network on the 1000 test images: %d %%' % (
            100 * correct / total))
    
    PATH = './skin_cancer_net.pth'
    torch.save(net.state_dict(), PATH)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.fc1 = nn.Linear(58*58*20, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.shape, " After 1st conv")
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape, " After 2nd conv")
        x = x.view(-1, 58*58*20)
        # print(x.shape, " After reshape")
        x = F.relu(self.fc1(x))
        # print(x.shape, " After fc1")
        x = F.relu(self.fc2(x))
        # print(x.shape, " After fc2")
        x = self.fc3(x)
        # print(x.shape, " After fc3")
        return x

if __name__ == "__main__":
    main()


Net(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=67280, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=3, bias=True)
)
Beginning training  1
[1,   101] loss: 0.400
[1,   201] loss: 0.307
[1,   301] loss: 0.341
[1,   401] loss: 0.289
[1,   501] loss: 0.254
[1,   601] loss: 0.308
Beginning training  2
[2,   101] loss: 0.309
[2,   201] loss: 0.241
[2,   301] loss: 0.211
[2,   401] loss: 0.209
[2,   501] loss: 0.202
[2,   601] loss: 0.210
Beginning training  3
[3,   101] loss: 0.223
[3,   201] loss: 0.219
[3,   301] loss: 0.171
[3,   401] loss: 0.217
[3,   501] loss: 0.177
[3,   601] loss: 0.195
Finished Training
544  correct
660  total
Accuracy of the network on the 1000 test images: 82 %


"\ntransform = transforms.Compose(\n    [transforms.ToTensor(),\n     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])\n\ntrainset = torchvision.datasets.CIFAR10(root='./data', train=True,\n                                        download=True, transform=transform)\ntrainloader = torch.utils.data.DataLoader(trainset, batch_size=4,\n                                          shuffle=True, num_workers=2)\n\ntestset = torchvision.datasets.CIFAR10(root='./data', train=False,\n                                       download=True, transform=transform)\ntestloader = torch.utils.data.DataLoader(testset, batch_size=4,\n                                         shuffle=False, num_workers=2)\n"